In [2]:
# Add parent directory
from sys import path as sysPath
sysPath.append('../')

# Add custom modules
sysPath.append('../Machine_Score/')

from Support_Code import general_module as gm
from Support_Code import info_module as im
from Image_Creator import main_image_creator as ic
from Machine_Score import main_machine_score as ms
from Machine_Score import direct_image_compare as dc
from Score_Analysis import main_score_analysis as sa
import main_SIMR as simr

# General modules
from os import path
import numpy as np
from matplotlib import pyplot as plt

sa.test()
gm.test()
im.test()
ic.test()
ms.test()
dc.test()
simr.test()

SA: Hi!  You're in Matthew's Main program for score analysis!
GM: Hi!  You're in Matthew's module for generally useful functions and classes
IM: Hi!  You're in Matthew Ogden's information module for SPAM
IC: Hi!  You're in Matthew's main code for all things image creation.
MS: Hi!  You're in Matthew's SIMR module for all things machine scoring images
DC: Hi!  You're in direct_image_compare.py
SIMR: Hi!  You're in Matthew's main program for all things galaxy collisions


In [3]:
tDir = '../targetDir'
tDir = gm.validPath(tDir)

tInfo = im.target_info_class( targetDir = tDir, printAll=False)
if tInfo.status == False:
    print("WARNING: target info class bad")
else:
    print("Target Good!")

# Get run info class
rInfo = tInfo.getRunInfo( rID='r00000' )

if rInfo.status == False:
    print("WARNING")
else:
    print("Run '%s' Good!"%rInfo.get('run_id'))

Target Good!
IM: run_info_class.__init__
	 - runDir:  /home/mbo2d/587722984435351614/zoo_merger_models/run_00000/
Run 'r00000' Good!


In [4]:
pLoc = '../param/start_param.json'
start_params = gm.readJson( pLoc )
gm.pprint(start_params)

for key in start_params:
    start_params[key]['scoreArg'] = start_params[key]['cmpArg']
    start_params[key].pop('cmpArg')
    start_params[key]['scoreArg']['']  
    gm.pprint(start_params[key])

tParams = tInfo.getImageParams()
gm.pprint(tParams)

{'cmpArg': {'cmpMethod': 'correlation'},
 'imgArg': {'comment': 'Basic image '
                       'parameter file',
            'galaxy_centers': {'px': '513',
                               'py': '514',
                               'sx': '309',
                               'sy': '608'},
            'image_size': {'height': 1024,
                           'width': '1024'},
            'name': 'zoo_0',
            'target_id': '587722984435351614'},
 'name': 'zoo_0',
 'scoreType': 'target',
 'simArg': {'nPts': '100k',
            'name': '100k'},
 'targetName': 'zoo_0',
 'zoo_0': {'cmpArg': {'cmpMethod': 'correlation'},
           'imgArg': {'comment': 'Basic '
                                 'image '
                                 'parameter '
                                 'file',
                      'galaxy_centers': {'px': '513',
                                         'py': '514',
                                         'sx': '309',
                               